In [2]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import time
import tracemalloc

In [3]:
# Nodurile din graf
cities = ['Bucuresti',
          'Craiova',
          'Brasov',
          'Pitesti',
          'Babadag',
          'Sibiu',
          'Cluj'
          ]


# Matricea de costuri
cost_matrix = np.array([
    [0, 3, 5, 10, 0, 0, 100],
    [0, 0, 0, 4, 0, 0, 0],
    [0, 0, 0, 4, 9, 3, 0],
    [0, 3, 0, 0, 2, 0, 0],
    [0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 4, 0, 5],
    [0, 0, 3, 0, 0, 0, 0]
])

start_city = 0  # Bucuresti
end_city = 4   # Craiova


print(f"Start: {cities[start_city]}")
print(f"End: {cities[end_city]}")

Start: Bucuresti
End: Babadag


In [4]:
class QLearningAgent:
    def __init__(self, cost_matrix, start, end, city_names=None, learning_rate=0.5, discount_factor=0.95, exploration_rate=0.5):

        self.cost_matrix = cost_matrix
        self.n_states = len(cost_matrix)
        self.start = start
        self.end = end
        self.city_names = city_names
        
        self.learning_rate = learning_rate    
        self.discount_factor = discount_factor   
        self.exploration_rate = exploration_rate 

        # Initializeaza Qtable cu zerouri
        self.q_table = np.zeros((self.n_states, self.n_states))
        
    def get_neighbors(self, state):
        # Returneaza lista de noduri vecine
        return [i for i in range(self.n_states) if self.cost_matrix[state][i] > 0]
    
    # Calculeaza recompensa pentru o actiune data
    def get_reward(self, state, action):
        # Costul din matricea de costuri
        cost = self.cost_matrix[state][action]
        #Daca actiunea duce la nodul final recompensa este mare
        if action == self.end:
            return 10000 / cost
        # Recompensa este negativa proportional cu costul
        else:
            return -100 * cost
    

    def choose_action(self, state, neighbors):

        # In functie de exploration rate ul dat ca parametru al fuctiei alege daca sa
        #mearga agentul pe un drum aleatoriu sau pe cel cu cea mai mare valoare Q

        #in cazul de exploration_rate=0.5 sansa de a alege un drum aleator este de 50%
        if np.random.random() < self.exploration_rate:
            #nod vecin random
            return np.random.choice(neighbors)
        else:
            #Lista cu valorile Q pentru nodurile vecine
            q_vals = [self.q_table[state][n] for n in neighbors]
            max_q = max(q_vals)
            #Lista cu nodurile vecine care au valoarea Q maxima
            best_actions = [n for i, n in enumerate(neighbors) if q_vals[i] == max_q]
            return np.random.choice(best_actions)
    
    def train(self, episodes=10000, max_steps=15):

        for ep in range(episodes):
            state = self.start
            steps = 0

            #Ruleaza pana cand ajunge la nodul final sau depaseste numarul maxim de pasi
            while state != self.end and steps < max_steps:
                neighbors = self.get_neighbors(state)
                #Daca nu are vecini iese din loop
                if not neighbors:
                    break

                action = self.choose_action(state, neighbors)
                reward = self.get_reward(state, action)

                #Vecinii urmatorului nod
                next_neighbors = self.get_neighbors(action)
                #Valoarea Q maxima dintre vecinii urmatorului nod
                max_next_q = max([self.q_table[action][n] for n in next_neighbors]) if next_neighbors else 0
                
                #Formula lui Bellman pentru actualizarea valorii Q
                self.q_table[state][action] += self.learning_rate * (reward + self.discount_factor * max_next_q - self.q_table[state][action])
                
                #Trece la urmatorul nod si se incrementeaza numarul de pasi
                state = action
                steps += 1
            
            #La fiecare 2000 de episoade scade exploration rate ul cu 10% pentru a favoriza exploatarea
            if ep % 2000 == 0 and ep > 0:
                self.exploration_rate = max(0.05, self.exploration_rate * 0.9)
                print(f"Episod {ep}/{episodes} | Exploration rate: {self.exploration_rate:.3f}")
    
    def get_optimal_path(self):
        path = [self.start]
        state = self.start
        #Folosim un set pentru a verifica daca un nod a fost deja vizitat
        visited = set([self.start])
        
        while state != self.end and len(path) < 15:
            neighbors = self.get_neighbors(state)
            unvisited = [n for n in neighbors if n not in visited]

            #Verificam doar nodurile care nu au fost vizitate
            if not unvisited:
                break
            #Cea mai buna valoarea Q a dintre nodurile nevizitate
            q_vals = [self.q_table[state][n] for n in unvisited]
            best = unvisited[np.argmax(q_vals)]

            #Adaugam nodul la path si il marcam ca vizitat
            path.append(best)
            visited.add(best)
            state = best

        return path
    
    def print_qtable(self):
        print("\nQTable:")
        for i in range(self.n_states):
            neighbors = self.get_neighbors(i)
            if neighbors:
                print(self.city_names[i], "->", end=" ")
                for n in neighbors:
                    print(f"{self.city_names[n]}:{round(self.q_table[i][n],2)}", end="  ")
                print()
    




In [6]:
## Antrenarea agentului Q-Learning

#Start la masurarea memoriei si a timpului
tracemalloc.start()
start_time = time.time()

agent = QLearningAgent(cost_matrix, start_city, end_city, city_names=cities,
                      learning_rate=0.5, discount_factor=0.95, exploration_rate=0.5)

#Antreneaza agentul pentru 10000 de episoade
agent.train(episodes=10000)

#Stop la masurarea memoriei si a timpului
end_time = time.time()
current_mem, peak_mem = tracemalloc.get_traced_memory()
tracemalloc.stop()

print("\n Training completed in", end_time - start_time, "seconds")

Episod 2000/10000 | Exploration rate: 0.450
Episod 4000/10000 | Exploration rate: 0.405
Episod 6000/10000 | Exploration rate: 0.365
Episod 8000/10000 | Exploration rate: 0.328

 Training completed in 1.3352813720703125 seconds


In [7]:
# Drum optim si cost
path = agent.get_optimal_path()
path_names = [cities[i] for i in path]
total_cost = sum(cost_matrix[path[i]][path[i+1]] for i in range(len(path)-1))

if path[-1] == end_city:
    print("Drum valid gasit")
else:
    print("Drum incomplet")

print(" -> ".join(path_names))
print(f"Cost total: {total_cost:.1f}")

# Performanta
print(f"Timp: {end_time - start_time:.4f} sec")
print(f"Memorie: {peak_mem / 1024:.2f} KB")

# Afiseaza Qtable
agent.print_qtable()


Drum valid gasit
Bucuresti -> Craiova -> Pitesti -> Babadag
Cost total: 9.0
Timp: 1.3353 sec
Memorie: 14.67 KB

QTable:
Bucuresti -> Craiova:3832.5  Brasov:3632.5  Pitesti:3750.0  Cluj:-6359.12  
Craiova -> Pitesti:4350.0  
Brasov -> Pitesti:4350.0  Babadag:1111.11  Sibiu:2683.83  
Pitesti -> Craiova:3832.5  Babadag:5000.0  
Sibiu -> Babadag:2500.0  Cluj:3140.88  
Cluj -> Brasov:3832.5  
